In [1]:
import pandas as pd
from pathlib import Path
from greensight.utils import DIR_ROOT

ModuleNotFoundError: No module named 'greensight'

In [8]:
path_soil_carbon = Path("../../share_point/data/JULES_outputs/Level_0_original_outputs/TW_UK_ERA5_BASE_V2_greenbelt16_ts_soil_carbon_interp_final__v1.xlsx")
path_veg_carbon = Path("../../share_point/data/JULES_outputs/Level_0_original_outputs/TW_UK_ERA5_BASE_V2_greenbelt16_ts_vegetation_carbon_interp_final__v1.xlsx")
assert path_soil_carbon.is_file()
assert path_veg_carbon.is_file()

In [9]:
df_soil_raw = pd.read_excel(path_soil_carbon)
df_veg_raw = pd.read_excel(path_veg_carbon)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.